<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/NGLextractionprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! apt update -q
! apt-get install -q openjdk-25-jdk-headless -q
! pip install neqsim -q
import neqsim
from neqsim.thermo.thermoTools import *
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

from neqsim.process import stream, expander, heater, compsplitter, separator, heatExchanger
from neqsim.thermo import fluid, printFrame
from neqsim.process import compsplitter, compressor, splitter, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle, setpoint, calculator


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Building dependency tree...
Reading state information...
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https

# NGL extraction process

**Watch the video:** [NGL Extraction Process on YouTube](https://www.youtube.com/watch?v=LrwlUReymlw)

# NGL extraction plant

In [17]:
#Feed gas
fluidcomposition = {'ComponentName':  ["nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane", "n-hexane", "C7", "C8", "C9"],
      'MolarComposition[-]':  [0.6, 0.007, 54.423, 8.0634, 4.8, 0.9, 1.7896, 0.6, 0.8, 0.8, 1.2, 1., 2.7571],
      'MolarMass[kg/mol]': [None,None, None,None,None,None,None,None,None,None,0.09832, 0.11227, 0.12627],
      'RelativeDensity[-]': [None,None, None,None,None,None,None,None,None,None, 0.737, 0.758, 0.775]
}
fluidcompositiondf = pd.DataFrame(fluidcomposition)
fluid1 = fluid_df(fluidcompositiondf, lastIsPlusFraction=False)
fluid1.setMixingRule('classic')

clearProcess()

streamfeed = stream(fluid1)

inletheater = heater(streamfeed)
inletheater.setOutPressure(110.0, 'bara')
inletheater.setOutTemperature(25.0, 'C')

separatorinlet = separator(inletheater.getOutletStream())

runProcess()

TypeError: stream() missing 1 required positional argument: 'thermoSystem'

In [ ]:
printFrame(separatorinlet.getGasOutStream().getFluid())

| 0                  | 1          | 2          | 3   | 4   | 5   | 6               |
|:-------------------|:-----------|:-----------|:----|:----|:----|:----------------|
|                    | total      | GAS        |     |     |     |                 |
| nitrogen           | 1.0086E-2  | 1.0086E-2  |     |     |     | [mole fraction] |
| CO2                | 9.31971E-5 | 9.31971E-5 |     |     |     | [mole fraction] |
| methane            | 8.30761E-1 | 8.30761E-1 |     |     |     | [mole fraction] |
| ethane             | 9.41011E-2 | 9.41011E-2 |     |     |     | [mole fraction] |
| propane            | 4.06942E-2 | 4.06942E-2 |     |     |     | [mole fraction] |
| i-butane           | 5.78628E-3 | 5.78628E-3 |     |     |     | [mole fraction] |
| n-butane           | 9.81506E-3 | 9.81506E-3 |     |     |     | [mole fraction] |
| i-pentane          | 2.2859E-3  | 2.2859E-3  |     |     |     | [mole fraction] |
| n-pentane          | 2.66089E-3 | 2.66089E-3 |     |     |     

In [ ]:
#NGL expander code

feedstream = separatorinlet.getGasOutStream()
feedstream.setFlowRate(10.0, 'MSm3/day')

pressureNGL = 50.6
onshoreplant_inlet_pressure = 110.0
onshoreplant_inlet_temperature = 20.0

clearProcess()

inletheater = heater(feedstream)
inletheater.setOutPressure(onshoreplant_inlet_pressure, 'bara')
inletheater.setOutTemperature(onshoreplant_inlet_temperature, 'C')

heatEx = heatExchanger(inletheater.getOutStream())
heatEx.setName("cross heat-exchanger")
heatEx.setGuessOutTemperature(273.15 - 30.0)
heatEx.setUAvalue(3000e3) #assumes an UA vlue of 3000 MW/m2K

expanderKarsto = expander(heatEx.getOutStream(0), pressureNGL)
expanderKarsto.setOutletPressure(pressureNGL)
expanderKarsto.setIsentropicEfficiency(0.75) # assume same efficiency as for compressors

scrubberNGL = separator(expanderKarsto.getOutletStream())

recyOut = stream(scrubberNGL.getGasOutStream().clone())

gasResycle = recycle2("gas resycle")
gasResycle.addStream(scrubberNGL.getGasOutStream())
gasResycle.setOutletStream(recyOut)

heatEx.setFeedStream(1, recyOut)

gasfromdeethanizer = stream(heatEx.getOutStream(1).clone())

gasmixer = mixer('gas mixer')
gasmixer.addStream(heatEx.getOutStream(1))
gasmixer.addStream(gasfromdeethanizer)

dryGasExportCompressor = compressor(gasmixer.getOutletStream())
dryGasExportCompressor.setOutletPressure(100.0, 'bara')
dryGasExportCompressor.setIsentropicEfficiency(0.75)

exportDryGas = stream(dryGasExportCompressor.getOutStream())

NGLvalve1 = valve(scrubberNGL.getLiquidOutStream())
NGLvalve1.setOutletPressure(30.0)

#FRACTIONATION PROCESS
deethanizer = distillationColumn(trays=5, reboil=True, condenser=False, name="deethanizer")
deethanizer.addFeedStream(NGLvalve1.getOutStream(),5)
deethanizer.getReboiler().setOutTemperature(273.15+105.0)
deethanizer.setTopPressure(30.0)
deethanizer.setBottomPressure(32.0)

compressorgasdeethanizer = compressor(deethanizer.getGasOutStream())
compressorgasdeethanizer.setOutletPressure(pressureNGL, 'bara')

nglGasResycle = recycle2("gas resycle 2")
nglGasResycle.addStream(compressorgasdeethanizer.getOutStream())
nglGasResycle.setOutletStream(gasfromdeethanizer)

valvetodebutanizer = valve(deethanizer.getLiquidOutStream())
valvetodebutanizer.setOutletPressure(14.0)

debutanizer = distillationColumn(trays=10, reboil=True, condenser=True, name="debutanizer")
debutanizer.addFeedStream(valvetodebutanizer.getOutStream(),9)
debutanizer.getCondenser().setRefluxRatio(0.1)
debutanizer.getCondenser().setTotalCondenser(True)
debutanizer.getReboiler().setOutTemperature(273.15+173.0);
debutanizer.setTopPressure(12.8)
debutanizer.setBottomPressure(15.0)

onshoreprocess = getProcess()

In [ ]:
thread = onshoreprocess.runAsThread()
thread.join(20 * 60 * 1000)   # 10 min maks tid

In [ ]:
#mass balance onshore process should be close to 0% lost
massbalanceonshore = (feedstream.getFlowRate('kg/hr') - exportDryGas.getFlowRate('kg/hr') - debutanizer.getLiquidOutStream().getFlowRate('kg/hr') - debutanizer.getGasOutStream().getFlowRate('kg/hr') )/feedstream.getFlowRate('kg/hr')

print('mass lost % ',massbalanceonshore*100)

mass lost %  3.6440353911302874


In [ ]:
print('feed gas rate ', feedstream.getFlowRate('MSm3/day'), ' MSm3/day')
printFrame(inletheater.getOutStream().getFluid())

feed gas rate  10.000000000000002  MSm3/day
| 0                  | 1          | 2          | 3          | 4   | 5   | 6               |
|:-------------------|:-----------|:-----------|:-----------|:----|:----|:----------------|
|                    | total      | GAS        | OIL        |     |     |                 |
| nitrogen           | 1.0086E-2  | 1.00966E-2 | 2.41688E-3 |     |     | [mole fraction] |
| CO2                | 9.31971E-5 | 9.32072E-5 | 8.58806E-5 |     |     | [mole fraction] |
| methane            | 8.30761E-1 | 8.3133E-1  | 4.16917E-1 |     |     | [mole fraction] |
| ethane             | 9.41011E-2 | 9.40521E-2 | 1.29704E-1 |     |     | [mole fraction] |
| propane            | 4.06942E-2 | 4.05952E-2 | 1.12665E-1 |     |     | [mole fraction] |
| i-butane           | 5.78628E-3 | 5.75945E-3 | 2.52889E-2 |     |     | [mole fraction] |
| n-butane           | 9.81506E-3 | 9.75412E-3 | 5.41231E-2 |     |     | [mole fraction] |
| i-pentane          | 2.2859E-3  | 

In [ ]:
print('propane in dry export gas ', exportDryGas.getFluid().getComponent('propane').getx()*100.0, ' mol%')
print('LPG production rate ', debutanizer.getGasOutStream().getFlowRate('idSm3/hr')*24.0 , ' Sm3/day')
print('C5+ rate ', debutanizer.getLiquidOutStream().getFlowRate('idSm3/hr')*24.0, ' Sm3/day')
print('temperature in NGL scrubber ', expanderKarsto.getOutStream().getTemperature('C'))
print('temperature LPG extraction reflux condenser ', debutanizer.getGasOutStream().getTemperature('C'))

print('duty deethanizer reboiler ', deethanizer.getReboiler().getDuty()/1e6, ' MW')
print('duty debutanizer reboiler ', debutanizer.getReboiler().getDuty()/1e6, ' MW')
print('duty debutanizer condenser ', debutanizer.getCondenser().getDuty()/1e6, ' MW')
propaneRecovery = 1.0 - exportDryGas.getFluid().getComponent('propane').getNumberOfmoles()/feedstream.getFluid().getComponent('propane').getNumberOfmoles()
ibutaneRecovery = 1.0 - exportDryGas.getFluid().getComponent('i-butane').getNumberOfmoles()/feedstream.getFluid().getComponent('i-butane').getNumberOfmoles()
nbutaneRecovery = 1.0 - exportDryGas.getFluid().getComponent('n-butane').getNumberOfmoles()/feedstream.getFluid().getComponent('n-butane').getNumberOfmoles()
print('propane recovery ' ,propaneRecovery)
print('i-butane recovery ' ,ibutaneRecovery)
print('n-butane recovery ' ,nbutaneRecovery)

propane in dry export gas  1.9855266986335707  mol%
LPG production rate  693.183570074583  Sm3/day
C5+ rate  501.9770540937247  Sm3/day
temperature in NGL scrubber  -46.984395643781795
temperature LPG extraction reflux condenser  34.38560175012759
GCV of exort gas  43.790305097431556  MJ/m3
duty deethanizer reboiler  10.694660990850409  MW
duty debutanizer reboiler  1.9829325708071843  MW
duty debutanizer condenser  -1.9610788358252031  MW
propane recovery  0.5347286575686505
i-butane recovery  0.7957713549171842
n-butane recovery  0.8576453078156216


## Export Gas and Liquid Products Quality

In [ ]:
print('export gas')
print('GCV of exort gas ', exportDryGas.GCV()/1e6, ' MJ/m3')
printFrame(exportDryGas.getFluid())

export gas
GCV of exort gas  43.790305097431556  MJ/m3
| 0                  | 1          | 2                  | 3   | 4   | 5   | 6               |
|:-------------------|:-----------|:-------------------|:----|:----|:----|:----------------|
|                    | total      | GAS                |     |     |     |                 |
| nitrogen           | 1.05768E-2 | 1.05768E-2         |     |     |     | [mole fraction] |
| CO2                | 9.76115E-5 | 9.76115E-5         |     |     |     | [mole fraction] |
| methane            | 8.71169E-1 | 8.71169E-1         |     |     |     | [mole fraction] |
| ethane             | 9.53081E-2 | 9.53081E-2         |     |     |     | [mole fraction] |
| propane            | 1.98553E-2 | 1.98553E-2         |     |     |     | [mole fraction] |
| i-butane           | 1.23923E-3 | 1.23923E-3         |     |     |     | [mole fraction] |
| n-butane           | 1.46522E-3 | 1.46522E-3         |     |     |     | [mole fraction] |
| i-pentane    

In [ ]:
#De-ethanizer results
print('gas from deethanizer')
printFrame(deethanizer.getGasOutStream().getFluid())
print('liquid from deethanizer')
printFrame(deethanizer.getLiquidOutStream().getFluid())

gas from deethanizer
| 0                  | 1          | 2                 | 3   | 4   | 5   | 6               |
|:-------------------|:-----------|:------------------|:----|:----|:----|:----------------|
|                    | total      | GAS               |     |     |     |                 |
| nitrogen           | 2.82015E-3 | 2.82015E-3        |     |     |     | [mole fraction] |
| CO2                | 1.74423E-4 | 1.74423E-4        |     |     |     | [mole fraction] |
| methane            | 6.65615E-1 | 6.65615E-1        |     |     |     | [mole fraction] |
| ethane             | 2.6608E-1  | 2.6608E-1         |     |     |     | [mole fraction] |
| propane            | 5.8057E-2  | 5.8057E-2         |     |     |     | [mole fraction] |
| i-butane           | 3.04265E-3 | 3.04265E-3        |     |     |     | [mole fraction] |
| n-butane           | 3.54453E-3 | 3.54453E-3        |     |     |     | [mole fraction] |
| i-pentane          | 3.28568E-4 | 3.28568E-4        |    

In [ ]:
#debutanizer results
print('LPG from top debutanizer ')
printFrame(debutanizer.getGasOutStream().getFluid())
print('C5+ from bottom debutanizer ')
printFrame(debutanizer.getLiquidOutStream().getFluid())

LPG from top debutanizer 
| 0                  | 1          | 2                  | 3   | 4   | 5   | 6               |
|:-------------------|:-----------|:-------------------|:----|:----|:----|:----------------|
|                    | total      | OIL                |     |     |     |                 |
| nitrogen           | 6.60258E-8 | 6.60258E-8         |     |     |     | [mole fraction] |
| CO2                | 4.8013E-6  | 4.8013E-6          |     |     |     | [mole fraction] |
| methane            | 9.14744E-4 | 9.14744E-4         |     |     |     | [mole fraction] |
| ethane             | 1.19354E-1 | 1.19354E-1         |     |     |     | [mole fraction] |
| propane            | 5.99902E-1 | 5.99902E-1         |     |     |     | [mole fraction] |
| i-butane           | 9.18175E-2 | 9.18175E-2         |     |     |     | [mole fraction] |
| n-butane           | 1.43183E-1 | 1.43183E-1         |     |     |     | [mole fraction] |
| i-pentane          | 2.02404E-2 | 2.02404E

In [ ]:
print('TVP of LPG product ', debutanizer.getGasOutStream().TVP(37.4, 'C'), 'bara')
print('TVP of C5+ product ', debutanizer.getLiquidOutStream().TVP(37.4, 'C'), 'bara')

TVP of LPG product  13.844836504310187 bara
TVP of C5+ product  0.8575966308296832 bara


In [ ]:
#De-ethanizer results
print('gas from deethanizer')
printFrame(deethanizer.getGasOutStream().getFluid())
print('liquid from deethanizer')
printFrame(deethanizer.getLiquidOutStream().getFluid())

gas from deethanizer
| 0                  | 1          | 2                 | 3   | 4   | 5   | 6               |
|:-------------------|:-----------|:------------------|:----|:----|:----|:----------------|
|                    | total      | GAS               |     |     |     |                 |
| nitrogen           | 2.82015E-3 | 2.82015E-3        |     |     |     | [mole fraction] |
| CO2                | 1.74423E-4 | 1.74423E-4        |     |     |     | [mole fraction] |
| methane            | 6.65615E-1 | 6.65615E-1        |     |     |     | [mole fraction] |
| ethane             | 2.6608E-1  | 2.6608E-1         |     |     |     | [mole fraction] |
| propane            | 5.8057E-2  | 5.8057E-2         |     |     |     | [mole fraction] |
| i-butane           | 3.04265E-3 | 3.04265E-3        |     |     |     | [mole fraction] |
| n-butane           | 3.54453E-3 | 3.54453E-3        |     |     |     | [mole fraction] |
| i-pentane          | 3.28568E-4 | 3.28568E-4        |    

In [ ]:
#debutanizer results
print('LPG from top debutanizer ')
printFrame(debutanizer.getGasOutStream().getFluid())
print('C5+ from bottom debutanizer ')
printFrame(debutanizer.getLiquidOutStream().getFluid())

LPG from top debutanizer 
| 0                  | 1          | 2                  | 3   | 4   | 5   | 6               |
|:-------------------|:-----------|:-------------------|:----|:----|:----|:----------------|
|                    | total      | OIL                |     |     |     |                 |
| nitrogen           | 6.60258E-8 | 6.60258E-8         |     |     |     | [mole fraction] |
| CO2                | 4.8013E-6  | 4.8013E-6          |     |     |     | [mole fraction] |
| methane            | 9.14744E-4 | 9.14744E-4         |     |     |     | [mole fraction] |
| ethane             | 1.19354E-1 | 1.19354E-1         |     |     |     | [mole fraction] |
| propane            | 5.99902E-1 | 5.99902E-1         |     |     |     | [mole fraction] |
| i-butane           | 9.18175E-2 | 9.18175E-2         |     |     |     | [mole fraction] |
| n-butane           | 1.43183E-1 | 1.43183E-1         |     |     |     | [mole fraction] |
| i-pentane          | 2.02404E-2 | 2.02404E